# 2. 그리드 서치 (GridSearch)
- 앞에서 모델의 일반화 성능을 추정하는 법을 배웠고, 이제 다음 단계로 매개변수를 튜닝하여 일반화 성능을 개선하겠음
- scikit-learn의 여러 알고리즘의 매개변수 설정에 대해 이야기했는데 매개변수를 조정하기 전에 그 매개변수의 의미를 이해하는 것이 중요함
- 모델에서 중요한 매개변수의 (일반화 성능을 최대로 높여주는) 값을 찾는 일은 어려운 작업이지만, 모든 모델과 데이터셋에서 해야 하는 필수적인 일임
- 많이 하는 작업이므로 scikit-learn에는 이를 위한 메서드가 준비되어 있음
- 가장 널리 사용하는 방법은 그리드 서치로서 관심 있는 매개변수들을 대상으로 가능한 모든 조합을 시도해보는 것
- SVC 파이썬 클래스에 구현된 RBF(Radial Basis Function) 커널 SVM을 사용
- 커널의 폭에 해당하는 gamma와 규제 매개변수 C가 중요
- 매개변수 C와 gamma에 0.001, 0.01, 0.1, 1, 10, 100 값을 적용해보겠음
- C와 gamma의 설정값이 각각 6개씩이니 조합의 수는 총 36개
- 모든 조합을 살펴보기 위해 SVM 매개변수 설정 테이블(그리드)을 만들 수 있음

## 2-1. 간단한 그리드 서치
- 두 매개변수 조합에 대해 분류기를 학습시키고 평가하는 간단한 그리드 서치를 for문을 사용해 만들 수 있음

In [1]:
# 간단한 그리드 서치 구현
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
print("훈련 세트의 크리: {}   테스트 세트의 크기: {}".format(X_train.shape[0], X_test.shape[0]))

best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
  for C in [0.001, 0.01, 0.1, 1, 10, 100]:
    # 매개변수의 각 조합에 대해 SVC를 훈련시킴
    svm = SVC(gamma=gamma, C=C)
    svm.fit(X_train, y_train)
    # 테스트 세트로 SVC를 평가
    score = svm.score(X_test, y_test)
    # 점수가 더 높으면 매개변수와 함께 기록
    if score > best_score:
      best_score = score
      best_parameters = {"C": C, "gamma": gamma}

print("최고 점수: {:.2f}".format(best_score))
print("최적 매개변수:", best_parameters)

훈련 세트의 크리: 112   테스트 세트의 크기: 38
최고 점수: 0.97
최적 매개변수: {'C': 100, 'gamma': 0.001}


## 2-2. 매개변수 과대적합과 검증 세트
- 앞의 결과를 보면 이 데이터셋에서 모델 정확도가 97%라고 보고할 수 있음
- 하지만 이런 주장은 다음과 같은 이유로 매우 낙관적인 (혹은 잘못된) 것일 수 있음
- 여러 가지 매개변수 값으로 많이 시도해보고 테스트 세트 정확도가 가장 높은 조합을 선택했음
- 하지만 이 정확도는 새로운 데이터에까지 이어지지 않을 수 있음
- 매개변수를 조정하기 위해 테스트 세트를 이미 사용했기 때문에 모델이 얼마나 좋은지 평가하는 데는 더 이상 사용할 수 없음
- 맨 처음 데이터를 훈련 세트와 테스트 세트로 나눈 이유와 같음
- 즉, 평가를 위해서는 모델을 만들 때 사용하지 않은 독립된 데이터셋이 필요
- 데이터를 다시 나눠서 세 개의 세트로 만들어 이 문제를 해결할 수 있음
- 훈련 세트로는 모델을 만들고, 검증 (또는 개발) 세트로는 모델의 매개변수를 선택하고, 테스트 세트로는 선택된 매개변수의 성능을 평가함

In [ ]:
mglearn